In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
products = pd.read_csv('products.csv')
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv')

In [3]:
transactions.describe()

,store_id,quantity,unit_price,margin
count,850548.000000,850548.000000,850548.000000,850548.000000
mean,25.544719,2.212695,5.567772,0.379868
std,14.419042,1.080688,2.894693,0.081023
min,1.000000,1.000000,1.490000,0.250000
25%,13.000000,1.000000,3.680000,0.350000
50%,26.000000,2.000000,4.700000,0.350000
75%,38.000000,3.000000,6.550000,0.450000
max,50.000000,10.000000,20.030000,0.500000


In [4]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850548 entries, 0 to 850547
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   transaction_id     850548 non-null  object 
 1   store_id           850548 non-null  int64  
 2   date               850548 non-null  object 
 3   product_id         850548 non-null  object 
 4   category           850548 non-null  object 
 5   quantity           850548 non-null  int64  
 6   unit_price         850548 non-null  float64
 7   is_loyalty_member  850548 non-null  bool   
 8   margin             850548 non-null  float64
dtypes: bool(1), float64(2), int64(2), object(4)
memory usage: 52.7+ MB


In [5]:
transactions.head()

,transaction_id,store_id,date,product_id,category,quantity,unit_price,is_loyalty_member,margin
0,T000001,13,2023-07-26 17:41:00,HOU_006,Household,2,6.75,True,0.45
1,T000001,13,2023-07-26 17:41:00,PER_028,Personal Care,1,8.59,True,0.50
2,T000001,13,2023-07-26 17:41:00,BEV_001,Beverages,1,3.59,True,0.50
3,T000001,13,2023-07-26 17:41:00,FRE_039,Fresh Produce,1,5.12,True,0.35
4,T000001,13,2023-07-26 17:41:00,FRO_037,Frozen Foods,3,5.70,True,0.40


In [6]:
transactions['date'] = pd.to_datetime(transactions['date'], dayfirst=True)
transactions['total_price'] = transactions['quantity'] * transactions['unit_price']

C:\Users\lewjj\AppData\Local\Temp\ipykernel_19976\2965138852.py:1: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  transactions['date'] = pd.to_datetime(transactions['date'], dayfirst=True)


In [7]:
store_info = transactions.copy()

In [8]:
store_info = store_info.merge(stores, on='store_id').merge(products, on='category')

In [9]:
store_info['is_weekend'] = store_info['date'].dt.dayofweek > 4
store_info['hour'] = store_info['date'].dt.hour
store_info['day'] = store_info['date'].dt.weekday

In [10]:
store_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850548 entries, 0 to 850547
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   transaction_id     850548 non-null  object        
 1   store_id           850548 non-null  int64         
 2   date               850548 non-null  datetime64[ns]
 3   product_id         850548 non-null  object        
 4   category           850548 non-null  object        
 5   quantity           850548 non-null  int64         
 6   unit_price         850548 non-null  float64       
 7   is_loyalty_member  850548 non-null  bool          
 8   margin_x           850548 non-null  float64       
 9   total_price        850548 non-null  float64       
 10  location_type      850548 non-null  object        
 11  store_size         850548 non-null  object        
 12  region             850548 non-null  object        
 13  avg_price          850548 non-null  float64 

In [11]:
store_info.head()

,transaction_id,store_id,date,product_id,category,quantity,unit_price,is_loyalty_member,margin_x,total_price,location_type,store_size,region,avg_price,margin_y,shelf_life_days,is_weekend,hour,day
0,T000001,13,2023-07-26 17:41:00,HOU_006,Household,2,6.75,True,0.45,13.50,Urban,Medium,North,6.99,0.45,365,False,17,2
1,T000001,13,2023-07-26 17:41:00,PER_028,Personal Care,1,8.59,True,0.50,8.59,Urban,Medium,North,7.99,0.50,365,False,17,2
2,T000001,13,2023-07-26 17:41:00,BEV_001,Beverages,1,3.59,True,0.50,3.59,Urban,Medium,North,3.49,0.50,180,False,17,2
3,T000001,13,2023-07-26 17:41:00,FRE_039,Fresh Produce,1,5.12,True,0.35,5.12,Urban,Medium,North,4.99,0.35,7,False,17,2
4,T000001,13,2023-07-26 17:41:00,FRO_037,Frozen Foods,3,5.70,True,0.40,17.10,Urban,Medium,North,5.99,0.40,180,False,17,2


In [12]:
# association rule mining

In [15]:
from mlxtend.frequent_patterns import apriori, association_rules

basket = store_info.groupby(['transaction_id', 'category'])['quantity'].sum().unstack()
basket = (basket > 0).astype(int)

frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

rules = association_rules(frequent_itemsets, num_itemsets=len(transactions), metric="lift", min_threshold=1.0)
print(rules)

C:\Users\lewjj\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


          antecedents                                        consequents  \
0         (Beverages)                                           (Bakery)   
1            (Bakery)                                        (Beverages)   
2            (Bakery)                                     (Canned Goods)   
3      (Canned Goods)                                           (Bakery)   
4            (Bakery)                                            (Dairy)   
...               ...                                                ...   
7467  (Fresh Produce)           (Snacks, Meat, Household, Personal Care)   
7468           (Meat)  (Snacks, Fresh Produce, Household, Personal Care)   
7469  (Personal Care)           (Snacks, Fresh Produce, Meat, Household)   
7470         (Snacks)    (Fresh Produce, Meat, Household, Personal Care)   
7471      (Household)       (Snacks, Fresh Produce, Meat, Personal Care)   

      antecedent support  consequent support   support  confidence      lift  \
0      

In [16]:
rules['zhangs_metric'] = (rules['confidence'] - rules['consequent support']) / (
    1 - rules['consequent support'])

high_lift_rules = rules[rules['lift'] > 1.2].sort_values(by='lift',ascending=False)
print("\nHigh-Lift Rules:")
print(high_lift_rules)

print("\nSummary Statistics:")
print(rules[['support', 'confidence', 'lift']].describe())


High-Lift Rules:
                                         antecedents  \
6355                             (Beverages, Snacks)   
6338                (Fresh Produce, Meat, Household)   
4630           (Fresh Produce, Bakery, Canned Goods)   
4643                               (Meat, Household)   
4526                       (Frozen Foods, Household)   
...                                              ...   
7200                                         (Dairy)   
5041                                         (Dairy)   
5012    (Fresh Produce, Meat, Bakery, Personal Care)   
7049                                         (Dairy)   
7024  (Fresh Produce, Meat, Household, Frozen Foods)   

                                            consequents  antecedent support  \
6355                   (Fresh Produce, Meat, Household)            0.161515   
6338                                (Beverages, Snacks)            0.134230   
4630                                  (Meat, Household)            0.133

In [17]:
# Store performance analysis
store_metrics = transactions.groupby('store_id').agg({
    'transaction_id': 'nunique', 
    'total_price': 'mean',   
    'is_loyalty_member': 'mean',  
    'quantity': 'sum'         
}).reset_index()

store_metrics = store_metrics.merge(stores, on='store_id')
store_metrics.head(10)

,store_id,transaction_id,total_price,is_loyalty_member,quantity,location_type,store_size,region
0,1,3936,12.332996,0.706748,36959,Rural,Medium,West
1,2,3897,12.246589,0.698712,36341,Urban,Small,West
2,3,3989,12.234082,0.696118,37163,Rural,Medium,West
3,4,4026,12.404657,0.693099,37791,Rural,Small,East
4,5,3926,12.335570,0.706928,36881,Urban,Medium,East
5,6,4043,12.432870,0.715677,38264,Urban,Large,East
6,7,4098,12.267266,0.701731,38429,Rural,Large,North
7,8,4018,12.389079,0.699754,37788,Suburban,Small,West
8,9,3953,12.269064,0.716701,36996,Rural,Large,East
9,10,3968,12.454123,0.711713,38013,Rural,Large,East


In [18]:
# Time-based analysis
hourly_patterns = store_info.groupby('hour').agg({
    'transaction_id': 'nunique',
    'total_price': 'mean'
}).reset_index()

weekly_patterns = store_info.groupby('day').agg({
    'transaction_id': 'nunique',
    'total_price': 'mean'
}).reset_index()

print("Hourly patterns: \n",hourly_patterns)
print("\nWeekly patterns:\n",weekly_patterns)

Hourly patterns: 
     hour  transaction_id  total_price
0      8           14318    12.410895
1      9           14271    12.382401
2     10           14440    12.281128
3     11           14334    12.294202
4     12           14394    12.322534
5     13           14157    12.341586
6     14           14108    12.357407
7     15           14194    12.376513
8     16           14281    12.291040
9     17           14420    12.233127
10    18           14127    12.323596
11    19           14345    12.342318
12    20           14398    12.327295
13    21           14213    12.329360

Weekly patterns:
    day  transaction_id  total_price
0    0           28071    11.987838
1    1           28596    11.959205
2    2           28901    11.996027
3    3           28391    11.979493
4    4           28377    11.992848
5    5           28726    13.164818
6    6           28938    13.207025


In [19]:
current_avg_transaction = transactions.groupby('transaction_id')['total_price'].sum().mean()
target_avg_transaction = current_avg_transaction * 1.10
total_transactions = transactions['transaction_id'].nunique()
potential_revenue_increase = (target_avg_transaction - current_avg_transaction) * total_transactions

print(f"Current Average Transaction Value: ${current_avg_transaction:.2f}")
print(f"Target Average Transaction Value: ${target_avg_transaction:.2f}")
print(f"Potential Annual Revenue Increase: ${potential_revenue_increase:.2f}")

Current Average Transaction Value: $52.43
Target Average Transaction Value: $57.68
Potential Annual Revenue Increase: $1048675.37


In [20]:
# roi
store_info['cost_per_unit'] = store_info['unit_price'] - store_info['margin_x']
store_info['total_cost'] = store_info['cost_per_unit'] * store_info['quantity']
store_info['roi'] = ((store_info['total_price'] - store_info['total_cost']) / store_info['total_cost']) * 100

roi_by_category = store_info.groupby('category')['roi'].mean()
roi_by_region = store_info.groupby('region')['roi'].mean()

print("ROI by Category:\n", roi_by_category)
print("\nROI by Region:\n", roi_by_region)

ROI by Category:
 category
Bakery           12.505421
Beverages        16.448975
Canned Goods     16.092586
Dairy             6.573206
Fresh Produce     7.418321
Frozen Foods      7.044271
Household         6.769650
Meat              2.324251
Personal Care     6.568135
Snacks           15.215166
Name: roi, dtype: float64

ROI by Region:
 region
East     9.219979
North    9.230815
South    9.219911
West     9.225383
Name: roi, dtype: float64
